In [1]:
import numpy as np

import spglib as spg

from hilde.io import read
from hilde.harmonic_analysis import HarmonicAnalysis
from hilde.structure.convert import to_spglib_cell
from hilde.helpers.numerics import clean_matrix
from hilde.helpers.lattice_points import get_commensurate_q_points

In [2]:
mat = 'gan'
primitive = read(f'{mat}/geometry.in')
supercell = read(f'{mat}/geometry.in.supercell')

In [3]:
ha = HarmonicAnalysis(primitive, supercell)

Set up harmonic analysis for Ga2N2:
.. found 25 (7) lattice points in 0.006s
.. found 25 (7) lattice points in 0.002s
** Force constants not set, your choice.
.. time elapsed: 0.057s


In [4]:
# Supercell matrix
P = supercell.cell @ primitive.get_reciprocal_cell().T

n_lp = np.round(np.linalg.det(P)).astype(int)
n_lp

25

In [5]:
my_mesh = np.round(ha.q_points @ supercell.cell.T).astype(int)
my_mesh, len(my_mesh)

(array([[ 0,  0,  0],
        [ 1,  1,  0],
        [ 1,  0,  1],
        [ 2,  1,  0],
        [ 1,  1,  1],
        [ 1,  2,  0],
        [ 2,  2,  0],
        [ 2,  1,  1],
        [ 1,  2,  1],
        [ 1,  3,  0],
        [ 3,  2,  0],
        [ 3,  1,  1],
        [ 3,  3, -1],
        [ 2,  2,  1],
        [ 2,  3,  0],
        [ 2,  4, -1],
        [ 3,  3,  0],
        [ 4,  2,  0],
        [ 4,  3, -1],
        [ 3,  4, -1],
        [ 4,  3,  0],
        [ 4,  4, -1],
        [ 3,  4,  0],
        [ 4,  4,  0],
        [ 4,  5, -1]]), 25)

In [6]:
n_qmesh = my_mesh.max(axis=0) - my_mesh.min(axis=0) + 1
n_qmesh

array([5, 6, 3])

In [7]:
my_mesh % n_qmesh

array([[0, 0, 0],
       [1, 1, 0],
       [1, 0, 1],
       [2, 1, 0],
       [1, 1, 1],
       [1, 2, 0],
       [2, 2, 0],
       [2, 1, 1],
       [1, 2, 1],
       [1, 3, 0],
       [3, 2, 0],
       [3, 1, 1],
       [3, 3, 2],
       [2, 2, 1],
       [2, 3, 0],
       [2, 4, 2],
       [3, 3, 0],
       [4, 2, 0],
       [4, 3, 2],
       [3, 4, 2],
       [4, 3, 0],
       [4, 4, 2],
       [3, 4, 0],
       [4, 4, 0],
       [4, 5, 2]])

In [8]:
ir_mapping, grid = spg.get_ir_reciprocal_mesh(n_qmesh, to_spglib_cell(primitive))
ir_grid = grid[np.unique(ir_mapping)]

In [9]:
match_list = -np.ones(n_lp, dtype=int)
for i1, q1 in enumerate(my_mesh % n_qmesh):
    for i2, q2 in enumerate(grid % n_qmesh):
        if np.linalg.norm(q1 - q2) < 1e-12:
            match_list[i1] = i2

assert len(np.unique(match_list)) == n_lp, (len(np.unique(match_list)), match_list)

In [10]:
match_list

array([ 0,  6, 31,  7, 36, 11, 12, 37, 41, 16, 13, 38, 78, 42, 17, 82, 18,
       14, 79, 83, 19, 84, 23, 24, 89])

In [11]:
dct = {}
for nn, ii in enumerate(np.unique(ir_mapping)):
    dct[ii] = nn

In [12]:
map_to_ir = np.array([dct[ii] for ii in ir_mapping[match_list]])
print(f'Number of q points reduced from\n  {len(map_to_ir)} to {len(np.unique(map_to_ir))}')

Number of q points reduced from
  25 to 18


In [13]:
ir_grid[np.unique(map_to_ir)]

array([[ 0,  0,  0],
       [ 1,  1,  0],
       [ 2,  1,  0],
       [ 1,  2,  0],
       [ 2,  2,  0],
       [-2,  2,  0],
       [-1,  2,  0],
       [ 1,  3,  0],
       [ 2,  3,  0],
       [ 1,  0,  1],
       [ 1,  1,  1],
       [ 2,  1,  1],
       [-2,  1,  1],
       [ 1,  2,  1],
       [ 2,  2,  1],
       [-2,  2,  1],
       [ 1,  3,  1],
       [ 2,  3,  1]], dtype=int32)